# Run Agent
- Implement a workflow to write a daily AI newsletter
- see README.md for details


In [1]:
import os
import yaml
import dotenv
import logging
import json
import yaml
from datetime import datetime
import time
import random
import glob
import pickle
import sqlite3

from pathlib import Path

import asyncio
import nest_asyncio

import pydantic
from pydantic import BaseModel, Field, RootModel
from typing import Dict, TypedDict, Type, List, Optional, Any, Iterable, Text
from dataclasses import dataclass, field
from enum import Enum

import numpy as np
import pandas as pd

import pandas as pd
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import hdbscan

import openai
from openai import AsyncOpenAI

import agents
from agents.exceptions import InputGuardrailTripwireTriggered
from agents import (Agent, Runner, Tool, OpenAIResponsesModel, 
                    ModelSettings, FunctionTool, InputGuardrail, GuardrailFunctionOutput,
                    SQLiteSession, set_default_openai_api, set_default_openai_client
                   )


import tenacity
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

from IPython.display import HTML, Image, Markdown, display

from log_handler import SQLiteLogHandler, setup_sqlite_logging, sanitize_error_for_logging
from config import LOGDB
from llm import LLMagent, LangfuseClient  # methods to apply prompts async to large batches
from db import Url 

from fetch import Fetcher # fetch news urls
from newsletter_state import NewsletterAgentState, StepStatus
from news_agent import NewsletterAgent


In [2]:
print(f"OpenAI:            {openai.__version__}")
print(f"OpenAI Agents SDK  {agents.__version__}")
print(f"Pydantic           {pydantic.__version__}")


OpenAI:            1.109.0
OpenAI Agents SDK  0.3.1
Pydantic           2.11.9


In [3]:
dotenv.load_dotenv()

# to run async in jupyter notebook
nest_asyncio.apply()

# verbose OpenAI console logging if something doesn't work
# logging.basicConfig(level=logging.DEBUG)
# openai_logger = logging.getLogger("openai")
# openai_logger.setLevel(logging.DEBUG)


In [4]:
# modules create a default logger, or we can pass this logger

def setup_logging(session_id: str = "default", db_path: str = "agent_logs.db") -> logging.Logger:
    """Set up logging to console and SQLite database."""

    # Create logger
    logging.basicConfig(level=logging.INFO)

    logger = logging.getLogger(f"NewsletterAgent.{session_id}")
    logger.setLevel(logging.INFO)

    # Clear any existing handlers
    logger.handlers.clear()

    # Console handler
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    console_formatter = logging.Formatter(
        '%(asctime)s | %(name)s | %(levelname)s | %(message)s',
        datefmt='%H:%M:%S'
    )
    console_handler.setFormatter(console_formatter)

    # SQLite handler
    sqlite_handler = SQLiteLogHandler(db_path)
    sqlite_handler.setLevel(logging.INFO)
    sqlite_formatter = logging.Formatter('%(message)s')
    sqlite_handler.setFormatter(sqlite_formatter)

    # Add handlers to logger
    logger.addHandler(console_handler)
    logger.addHandler(sqlite_handler)

    # Prevent propagation to root logger
    logger.propagate = False

    return logger

logger = setup_logging("newsletter_agent", "test_logs.db")

# Log some test messages
logger.info("Test info message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.warning("Test warning message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.error("Test error message", extra={
    'step_name': 'error_step',
    'agent_session': 'demo_session'
})

sanitize_error_for_logging("log with some bad stuff for the filter: sk-proj-123456789012345678901234567890123456789012345678")

21:03:41 | NewsletterAgent.newsletter_agent | INFO | Test info message
21:03:41 | NewsletterAgent.newsletter_agent | WARNING | Test warning message
21:03:41 | NewsletterAgent.newsletter_agent | ERROR | Test error message


'log with some bad stuff for the filter: [API_KEY_REDACTED]'

# Run Agent Worfklow

In [5]:
print("🚀 Creating NewsletterAgent...")

do_download=True
process_since=None
# process_since='2025-10-05 18:30:00'

try:
    # set up state
    session_id = 'test_newsletter_20251012203454309410'
    step_name = 'step_04_extract_summaries'
#     del session_id, step_name
except Exception as e:
    print(e)

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set")

# Set up OpenAI client for the agents SDK
set_default_openai_client(AsyncOpenAI(api_key=api_key))

# Create agent with persistent state
if 'session_id' in vars():
    # load state from db for session_id and state
    print("session_id is defined")
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, 
                                 db_path="newsletter_agent.db", 
                                 do_download=do_download,
                                 process_since=process_since,
                                 verbose=False
                                )
    state = state.load_from_db(step_name)
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=True, timeout=30)    
else:
    # create new session
    print("session_id is not defined")
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S%f")    
    session_id = f"test_newsletter_{timestamp}"
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, 
                                 db_path="newsletter_agent.db",
                                 do_download=do_download,
                                 process_since=process_since,
                                 verbose=False
                                ) 
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=False, timeout=30)
    state.serialize_to_db("initialize")

21:03:58 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Using provided state with 64 articles


🚀 Creating NewsletterAgent...
session_id is defined
test_newsletter_20251012203454309410
Initialized NewsletterAgent with persistent state and 9-step workflow
Session ID: test_newsletter_20251012203454309410


In [6]:
agent.state.get_status()


{'headlines': {'total': 64},
 'sources': {'config_file': 'sources.yaml', 'loaded_sources': 0},
 'topics': {'cluster_topics': 0, 'topics': []},
 'workflow': {'current_step': 'step_05_rate_articles',
  'workflow_complete': False,
  'workflow_status': 'started',
  'workflow_status_message': '',
  'progress_percentage': 44.44444444444444,
  'max_edits': 2,
  'concurrency': 16,
  'do_download': True,
  'process_since': None},
 'processing': {'topic_clusters': 0,
  'newsletter_sections': 0,
  'final_newsletter_length': 0}}

In [7]:
# User prompt to run workflow
user_prompt = "Show the workflow status"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


📝 User prompt: 'Show the workflow status'


20:35:09 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Starting check_workflow_status
20:35:09 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Completed check_workflow_status


⏱️  Total execution time: 8.74s
📊 Final result:
Current workflow status:
- Progress: 0.0% (0/9 complete)
- 0 complete, 0 started, 0 failed, 9 not started
- Next step: Step 1 — Gather URLs (Fetch Urls)

Detailed step states:
1. Fetch Urls: not_started
2. Filter Urls: not_started
3. Download Articles: not_started
4. Extract Summaries: not_started
5. Rate Articles: not_started
6. Cluster By Topic: not_started
7. Select Sections: not_started
8. Draft Sections: not_started
9. Finalize Newsletter: not_started

Would you like me to start Step 1 (gather URLs) or run all steps to create the newsletter?


In [8]:
# User prompt to run a workflow step
user_prompt = "Run step 1, fetch urls"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)



📝 User prompt: 'Run step 1, fetch urls'


20:35:19 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Starting check_workflow_status
20:35:19 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Completed check_workflow_status
20:35:21 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Starting Step 1: Gather URLs
20:35:21 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Cleaning download/sources: 
20:35:21 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Successfully cleaned download/sources
20:35:21 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Cleaning download/text: 
20:35:21 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Successfully cleaned download/text
20:35:21 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Cleaning download/html: 
20:35:21 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Successfully cleaned download/html
2025-10-12 20:35:21,288 - fetcher_5279693584 - INFO - [fetcher_in

▶ Starting Step 1: step_01_fetch_urls


2025-10-12 20:35:21,554 - fetcher_5279693584 - INFO - [fetch_rss] RSS fetch successful for HackerNoon: 50 articles
2025-10-12 20:35:21,555 - fetcher_5279693584 - INFO - [fetch_html] Fetching HTML from Techmeme: https://www.techmeme.com/river
2025-10-12 20:35:21,625 - fetcher_5279693584 - INFO - [fetch_rss] RSS fetch successful for Ars Technica: 20 articles
2025-10-12 20:35:21,627 - fetcher_5279693584 - INFO - [fetch_rss] Fetching RSS from The Register: https://www.theregister.com/software/ai_ml/headlines.atom
2025-10-12 20:35:21,678 - fetcher_5279693584 - INFO - [fetch_rss] RSS fetch successful for Hacker News: 30 articles
2025-10-12 20:35:21,679 - fetcher_5279693584 - INFO - [fetch_html] Fetching HTML from The Verge: https://www.theverge.com/ai-artificial-intelligence
2025-10-12 20:35:21,752 - fetcher_5279693584 - INFO - [fetch_rss] RSS fetch successful for The Register: 50 articles
2025-10-12 20:35:21,761 - fetcher_5279693584 - INFO - [fetch_html] Fetching HTML from VentureBeat: http

2025-10-12 20:35:23,330 - fetcher_5279693584 - INFO - scraping https://feedly.com/i/aiFeeds?options=eyJsYXllcnMiOlt7InBhcnRzIjpbeyJpZCI6Im5scC9mL3RvcGljLzMwMDAifV0sInNhbGllbmNlIjoibWVudGlvbiIsInNlYXJjaEhpbnQiOiJ0ZWNobm9sb2d5IiwidHlwZSI6Im1hdGNoZXMifV0sImJ1bmRsZXMiOnsiYWxsRmVlZGx5TW9kZSI6W3sidHlwZSI6InN0cmVhbSIsImlkIjoidXNlci82MmVlYmI5Zi03MTUxLTRmOWEtYThjNy05YTU3YjgyMDUzMDgvY2F0ZWdvcnkvZ2xvYmFsLmFsbCJ9LHsidHlwZSI6InB1YmxpY2F0aW9uQnVja2V0IiwiaWQiOiJkaXNjb3Zlcnk6YWxsLXRvcGljcyIsInRpZXIiOiJ0aWVyMSJ9XSwiY3VzdG9tTW9kZSI6W3sidHlwZSI6InN0cmVhbSIsImlkIjoidXNlci82MmVlYmI5Zi03MTUxLTRmOWEtYThjNy05YTU3YjgyMDUzMDgvY2F0ZWdvcnkvZ2xvYmFsLmFsbCJ9XSwiYm9hcmRNb2RlIjpbeyJ0eXBlIjoic3RyZWFtIiwiaWQiOiJ1c2VyLzYyZWViYjlmLTcxNTEtNGY5YS1hOGM3LTlhNTdiODIwNTMwOC90YWcvZ2xvYmFsLmFsbCJ9XSwiYW5ub3RhdGVkTW9kZSI6W3sidHlwZSI6InN0cmVhbSIsImlkIjoidXNlci82MmVlYmI5Zi03MTUxLTRmOWEtYThjNy05YTU3YjgyMDUzMDgvdGFnL2dsb2JhbC5hbm5vdGF0ZWQifV19LCJyZWZpbmVNb2RlIjoiY3VzdG9tTW9kZSIsImxhbmd1YWdlcyI6WyJlbiJdfQ to download/sources
2025-10-1

...

2025-10-12 20:35:49,919 - fetcher_5279693584 - DEBUG - Parsed datetime: 2025-10-13 01:35:49
2025-10-12 20:35:49,919 - fetcher_5279693584 - DEBUG - Added UTC timezone: 2025-10-13 01:35:49+00:00
2025-10-12 20:35:49,919 - fetcher_5279693584 - DEBUG - Converted to UTC: 2025-10-13 01:35:49+00:00
2025-10-12 20:35:49,919 - fetcher_5279693584 - DEBUG - Formatted last_updated: 2025-10-13T01:35:49Z
2025-10-12 20:35:49,920 - fetcher_5279693584 - INFO - Saving HTML to download/sources/Reddit.html
2025-10-12 20:35:49,925 - fetcher_5279693584 - INFO - [fetch_html] Parsing HTML file: download/sources/Reddit.html
2025-10-12 20:35:49,959 - fetcher_5279693584 - INFO - [fetch_html] Parsed HTML file: download/sources/Reddit.html
2025-10-12 20:35:49,960 - fetcher_5279693584 - INFO - [fetch_html] HTML fetch successful for Reddit: 61 articles
2025-10-12 20:35:51,419 - fetcher_5279693584 - DEBUG - performed human like actions
2025-10-12 20:35:51,419 - fetcher_5279693584 - INFO - Scrolling Feedly_AI (1/3)
2025

,source,url
0,Ars Technica,20
1,Bloomberg,31
2,Business Insider,15
3,FT,49
4,Feedly AI,71
5,Hacker News,30
6,HackerNoon,50
7,New York Times,20
8,NewsAPI,21
9,Reddit,51


,source,title,url,published,rss_summary,id
0,Ars Technica,OpenAI will stop saving most ChatGPT users’ de...,https://arstechnica.com/tech-policy/2025/10/op...,"Fri, 10 Oct 2025 14:58:29 +0000",Court ends controversial order forcing OpenAI ...,0
1,Ars Technica,“Like putting on glasses for the first time”—h...,https://arstechnica.com/science/2025/10/like-p...,"Fri, 10 Oct 2025 11:00:24 +0000",AI is “comically good” at detecting small eart...,1
2,Ars Technica,AI models can acquire backdoors from surprisin...,https://arstechnica.com/ai/2025/10/ai-models-c...,"Thu, 09 Oct 2025 22:03:21 +0000","Anthropic study suggests ""poison"" training att...",2
3,Ars Technica,Bank of England warns AI stock bubble rivals 2...,https://arstechnica.com/ai/2025/10/bank-of-eng...,"Wed, 08 Oct 2025 21:18:30 +0000",Central bank says market concentration hasn't ...,3
4,Ars Technica,Vandals deface ads for AI necklaces that liste...,https://arstechnica.com/tech-policy/2025/10/va...,"Wed, 08 Oct 2025 16:07:29 +0000",Critics attacked subway ads to defend human fr...,4
...,...,...,...,...,...,...
667,NewsAPI,US government launches major pursuit of revolu...,https://www.yahoo.com/news/articles/us-governm...,2025-10-11T21:15:00Z,NaN,667
668,NewsAPI,Frustrated Job Seekers Try to Outsmart AI Résu...,https://www.newser.com/story/376599/frustrated...,2025-10-11T22:15:00Z,NaN,668
669,NewsAPI,"Next time you consult an AI chatbot, remember ...",https://cnalifestyle.channelnewsasia.com/welln...,2025-10-11T23:14:38Z,NaN,669
670,NewsAPI,Batman/Superman: World's Finest #44 Preview: R...,https://bleedingcool.com/comics/batman-superma...,2025-10-11T22:26:04Z,NaN,670


20:35:58 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Completed Step 1: Gathered 771 articles
20:35:59 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Starting check_workflow_status
20:35:59 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Completed check_workflow_status


⏱️  Total execution time: 48.06s
📊 Final result:
Step 1 completed: I fetched URLs.

Summary:
- Step 1 (Fetch Urls): complete
- Articles gathered: 672 (from 17 RSS sources)
- Next step: Step 2 — Filter URLs (AI-related)

Shall I proceed to Step 2 (filter to AI-related content) now?


In [9]:
pd.DataFrame(state.headline_data) 


,source,title,url,published,rss_summary,id
0,Ars Technica,OpenAI will stop saving most ChatGPT users’ de...,https://arstechnica.com/tech-policy/2025/10/op...,"Fri, 10 Oct 2025 14:58:29 +0000",Court ends controversial order forcing OpenAI ...,0
1,Ars Technica,“Like putting on glasses for the first time”—h...,https://arstechnica.com/science/2025/10/like-p...,"Fri, 10 Oct 2025 11:00:24 +0000",AI is “comically good” at detecting small eart...,1
2,Ars Technica,AI models can acquire backdoors from surprisin...,https://arstechnica.com/ai/2025/10/ai-models-c...,"Thu, 09 Oct 2025 22:03:21 +0000","Anthropic study suggests ""poison"" training att...",2
3,Ars Technica,Bank of England warns AI stock bubble rivals 2...,https://arstechnica.com/ai/2025/10/bank-of-eng...,"Wed, 08 Oct 2025 21:18:30 +0000",Central bank says market concentration hasn't ...,3
4,Ars Technica,Vandals deface ads for AI necklaces that liste...,https://arstechnica.com/tech-policy/2025/10/va...,"Wed, 08 Oct 2025 16:07:29 +0000",Critics attacked subway ads to defend human fr...,4
...,...,...,...,...,...,...
667,NewsAPI,US government launches major pursuit of revolu...,https://www.yahoo.com/news/articles/us-governm...,2025-10-11T21:15:00Z,NaN,667
668,NewsAPI,Frustrated Job Seekers Try to Outsmart AI Résu...,https://www.newser.com/story/376599/frustrated...,2025-10-11T22:15:00Z,NaN,668
669,NewsAPI,"Next time you consult an AI chatbot, remember ...",https://cnalifestyle.channelnewsasia.com/welln...,2025-10-11T23:14:38Z,NaN,669
670,NewsAPI,Batman/Superman: World's Finest #44 Preview: R...,https://bleedingcool.com/comics/batman-superma...,2025-10-11T22:26:04Z,NaN,670


In [10]:
countdf = pd.DataFrame(state.headline_data) \
    .groupby("source") \
    .count()[["id"]] \
    .reset_index() \
    .rename(columns={'id': 'count'}) \
    .sort_values("count", ascending=False)
countdf 


,source,count
10,Techmeme,168
4,Feedly AI,71
9,Reddit,51
6,HackerNoon,50
12,The Register,50
3,FT,49
1,Bloomberg,31
5,Hacker News,30
13,The Verge,26
11,The Guardian,23


In [11]:
# Run tool directly without LLM processing an input prompt or results
# user_prompt = "Run step 2, filter urls"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("filter_urls")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


20:36:04 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Starting Step 2: Filter URLs
20:36:04 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | 🔍 Filtering 672 headlines...
20:36:04 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | 🔄 Checking for duplicates (all urls without date restrictions)
20:36:04 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | 🔍 Filtering 672 articles for dupes.


▶ Starting Step 2: step_02_filter_urls
checking https://arstechnica.com/tech-policy/2025/10/openai-no-longer-forced-to-save-deleted-chats-but-some-users-still-affected/
found url, no cutoff set - treating as duplicate
checking https://arstechnica.com/science/2025/10/like-putting-on-glasses-for-the-first-time-how-ai-improves-earthquake-detection/
found url, no cutoff set - treating as duplicate
checking https://arstechnica.com/ai/2025/10/ai-models-can-acquire-backdoors-from-surprisingly-few-malicious-documents/
found url, no cutoff set - treating as duplicate
checking https://arstechnica.com/ai/2025/10/bank-of-england-warns-ai-stock-bubble-rivals-2000-dotcom-peak/
found url, no cutoff set - treating as duplicate
checking https://arstechnica.com/tech-policy/2025/10/vandals-deface-ads-for-ai-necklaces-that-listen-to-all-your-conversations/
found url, no cutoff set - treating as duplicate
checking https://arstechnica.com/ai/2025/10/insurers-balk-at-paying-out-huge-settlements-for-claims-ag

url not found - inserting new URL
checking https://restofworld.org/2025/young-people-in-china-are-embracing-ai-therapy/
url not found - inserting new URL
checking https://www.seattletimes.com/business/they-had-money-problems-they-turned-to-chatgpt-for-solutions/
url not found - inserting new URL
checking https://flip.it/N9zE-O
url not found - inserting new URL
checking https://www.fool.com/investing/2025/10/12/3-millionaire-maker-artificial-intelligence-ai-sto/
url not found - inserting new URL
checking https://www.fool.com/investing/2025/10/12/open-ai-stock-spacex-stock-xai-stock-nvda-stock/
url not found - inserting new URL
checking https://indiandefencereview.com/artificial-intelligence-uncovers-5000-year-old-civilizations-buried-beneath-the-worlds-largest-and-harshest-desert/
url not found - inserting new URL
checking https://hackernoon.com/10-12-2025-newsletter
url not found - inserting new URL
checking https://www.benzinga.com/markets/tech/25/10/48167448/oracles-hype-machine-prep

20:36:04 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | URL deduplication: 518 duplicates filtered, 154 new URLs remain
20:36:04 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | 🔍 Filtering 672 headlines for AI relevance using LLM...
INFO:llm:Initialized LangfuseClient


url not found - inserting new URL
checking https://www.reddit.com/r/Futurology/comments/1o4haqg/ai_enabled_klarna_to_halve_its_workforcenow_the/
url not found - inserting new URL
checking https://www.techspot.com/news/109824-china-tightens-rare-earth-exports-threatening-global-chip.html
found url, no cutoff set - treating as duplicate
checking https://www.reddit.com/r/technology/comments/1o4c4jt/china_tightens_rare_earth_exports_threatening/
found url, no cutoff set - treating as duplicate
checking https://www.theguardian.com/commentisfree/2025/oct/10/sam-altman-copyright-ai-sora-2-video-generator
found url, no cutoff set - treating as duplicate
checking https://www.reddit.com/r/technology/comments/1o4mdmh/its_sam_altman_the_man_who_stole_the_rights_from/
found url, no cutoff set - treating as duplicate
checking https://fortune.com/2025/10/10/a-3-person-policy-non-profit-that-worked-on-californias-ai-safety-law-is-publicly-accusing-openai-of-intimidation-tactics/
found url, no cutoff s

INFO:llm:Successfully retrieved prompt 'newsagent/filter_urls' from Langfuse
INFO:llm:Parsed prompt 'newsagent/filter_urls': model=gpt-4.1-mini, system_len=459, user_len=954
20:36:04 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Processing 7 chunks with concurrency 16


concurrency:  16


20:36:18 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Completed Step 2: 64 AI-related articles, 518 duplicates removed


⏱️  Total execution time: 13.56s
📊 Final result:
✅ Step 2 step_02_filter_urls completed successfully! Removed 518 duplicate URLs, classified 154 new articles, found 64 AI-related.


In [12]:
# User prompt to run workflow
# user_prompt = "Run step 3, download full articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("download_articles")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

20:36:18 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Starting Step 3: Download Articles
20:36:18 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Starting concurrent scraping of 64 AI-related articles


▶ Starting Step 3: step_03_download_articles


20:36:18 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Launching browser for 64 URLs with 16 concurrent workers
20:36:18 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Worker 0 fetching 1 of 64 https://www.benzinga.com/markets/tech/25/10/48167448/oracles-hype-machine-preps-for-ai-world-2025
20:36:18 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | scrape_url(https://www.benzinga.com/markets/tech/25/10/48167448/oracles-hype-machine-preps-for-ai-world-2025)
20:36:18 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | scraping https://www.benzinga.com/markets/tech/25/10/48167448/oracles-hype-machine-preps-for-ai-world-2025 to download/html
20:36:18 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Downloading https://www.benzinga.com/markets/tech/25/10/48167448/oracles-hype-machine-preps-for-ai-world-2025
20:36:18 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Worker 1 fetching 2 of 64 https://www

20:36:18 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | scraping https://www.joblo.com/george-miller-on-ai/ to download/html
20:36:18 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Downloading https://www.joblo.com/george-miller-on-ai/
20:36:18 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Worker 11 fetching 13 of 64 https://www.usatoday.com/story/money/careers/2025/10/12/advice-to-students-in-a-work-world-with-ai/86611972007/
20:36:18 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | scrape_url(https://www.usatoday.com/story/money/careers/2025/10/12/advice-to-students-in-a-work-world-with-ai/86611972007/)
20:36:18 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | scraping https://www.usatoday.com/story/money/careers/2025/10/12/advice-to-students-in-a-work-world-with-ai/86611972007/ to download/html
20:36:18 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Downloading https://www.usatoday.com

20:37:00 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | scraping https://finance.yahoo.com/news/varonis-builds-ai-security-portfolio-181415851.html to download/html
20:37:00 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Downloading https://finance.yahoo.com/news/varonis-builds-ai-security-portfolio-181415851.html
20:37:02 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Response: 200
20:37:04 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Saving HTML to download/html/Frustrated_Job_Seekers_Try_to_Outsmart_AI_Résumé_Bots.html
20:37:04 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Worker 4 completed https://www.newser.com/story/376599/frustrated-job-seekers-try-to-outsmart-ai-resume-bots.html with status: 200
20:37:04 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Worker 4 fetching 20 of 64 https://comicbookmovie.com/sci_fi/chris-pratt-is-at-the-mercy-of-rebecca-fergusons-ai-judge-in-fir

...

concurrency:  16


20:40:46 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Inserted 5 new site records
20:40:46 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Updated 64 URL records with final URLs


Starting with 64 rows...
Processing 64 files...
Reading and truncating files to 8192 tokens using text-embedding-3-large tokenizer...
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Getting embeddings for 58 texts...


20:40:49 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Completed Step 3: Downloaded 58 articles


Creating indexed similarity matrix...
Finding pairs with similarity > 0.925...
Filtering dataframe...
Removing 0 rows due to high similarity 
Final dataframe has 64 rows (removed 0 rows)
⏱️  Total execution time: 271.21s
📊 Final result:
✅ Step 3 step_03_download_articles completed successfully! Downloaded 58 AI-related articles with 91% success rate.
📊 Average article length: 5894 characters
🔗 Content stored in persistent state.


In [13]:
headline_df = state.headline_df
with pd.option_context('display.max_columns', None, 'display.width', None, 'display.max_colwidth', None  ):
    display(headline_df[['id', 'source', 'title', 'status', 'final_url']].loc[headline_df["status"]!=200])




,id,source,title,status,final_url
58,21,Bloomberg,"World Economy Faces Triple Risk of Tariffs, AI Bubble and Soaring Debt",skipped,https://www.bloomberg.com/news/articles/2025-10-12/world-economy-faces-triple-risk-of-tariffs-ai-bubble-and-soaring-debt
59,595,WSJ,Why AI Will Widen the Gap Between Superstars and Everybody Else,skipped,https://www.wsj.com/lifestyle/workplace/ai-workplace-tensions-what-to-do-c45f6b51
60,594,WSJ,"Plus. the “cobot” revolution, Musk’s xAI supercomputer bonanza and how AI is complicating career advice, in this edition of the Technology newsletter.",skipped,https://www.wsj.com/tech/ai/amd-amps-up-chip-war-274560d5
61,342,Techmeme,"Sources: Apple glasses may run full visionOS when paired with Mac, or a lighter UI with iPhone; Apple may unveil M5 iPad, MacBook Pro, and Vision Pro this week",skipped,https://www.bloomberg.com/news/newsletters/2025-10-12/inside-apple-s-glasses-pivot-when-apple-is-launching-m5-macbook-pro-ipad-pro
62,298,Reddit,"The AI bubble is 17 times the size of the dot-com frenzy — and four times the subprime bubble, analyst says",401,https://www.marketwatch.com/story/the-ai-bubble-is-17-times-the-size-of-the-dot-com-frenzy-this-analyst-argues-046e7c5c
63,20,Bloomberg,AI Brings Korea’s Struggling Film Industry Hopes and Anxiety,skipped,https://www.bloomberg.com/news/articles/2025-10-12/ai-brings-korea-s-struggling-film-industry-hopes-and-anxiety


In [14]:
headline_df = state.headline_df
with pd.option_context('display.max_columns', None, 'display.width', None, 'display.max_colwidth', None  ):
    display(headline_df[['id', 'source', 'title', 'status', 'final_url']].loc[headline_df["html_path"]==""])



,id,source,title,status,final_url
58,21,Bloomberg,"World Economy Faces Triple Risk of Tariffs, AI Bubble and Soaring Debt",skipped,https://www.bloomberg.com/news/articles/2025-10-12/world-economy-faces-triple-risk-of-tariffs-ai-bubble-and-soaring-debt
59,595,WSJ,Why AI Will Widen the Gap Between Superstars and Everybody Else,skipped,https://www.wsj.com/lifestyle/workplace/ai-workplace-tensions-what-to-do-c45f6b51
60,594,WSJ,"Plus. the “cobot” revolution, Musk’s xAI supercomputer bonanza and how AI is complicating career advice, in this edition of the Technology newsletter.",skipped,https://www.wsj.com/tech/ai/amd-amps-up-chip-war-274560d5
61,342,Techmeme,"Sources: Apple glasses may run full visionOS when paired with Mac, or a lighter UI with iPhone; Apple may unveil M5 iPad, MacBook Pro, and Vision Pro this week",skipped,https://www.bloomberg.com/news/newsletters/2025-10-12/inside-apple-s-glasses-pivot-when-apple-is-launching-m5-macbook-pro-ipad-pro
62,298,Reddit,"The AI bubble is 17 times the size of the dot-com frenzy — and four times the subprime bubble, analyst says",401,https://www.marketwatch.com/story/the-ai-bubble-is-17-times-the-size-of-the-dot-com-frenzy-this-analyst-argues-046e7c5c
63,20,Bloomberg,AI Brings Korea’s Struggling Film Industry Hopes and Anxiety,skipped,https://www.bloomberg.com/news/articles/2025-10-12/ai-brings-korea-s-struggling-film-industry-hopes-and-anxiety


In [15]:
# User prompt to run workflow
# user_prompt = "Run step 4, Summarize articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("extract_summaries")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

20:40:49 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Processing 64 AI articles for summarization
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/extract_summaries' from Langfuse
INFO:llm:Parsed prompt 'newsagent/extract_summaries': model=gpt-4.1-mini, system_len=1273, user_len=43
20:40:49 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Using model 'gpt-4.1-mini' for summarization
20:40:49 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Starting summarization for 64 articles
20:40:49 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Processing 64 chunks with concurrency 16


▶ Starting Step 4: step_04_extract_summaries
concurrency:  16


20:41:00 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Extracting metadata from HTML files for 64 articles
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/item_distiller' from Langfuse
INFO:llm:Parsed prompt 'newsagent/item_distiller': model=gpt-5-mini, system_len=1365, user_len=151
20:41:02 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Processing 64 chunks with concurrency 16


concurrency:  16
⏱️  Total execution time: 53.83s
📊 Final result:
✅ Step 4 step_04_extract_summaries completed successfully! Generated AI-powered summaries for 64/64 articles.
💾 Summaries stored in headline DataFrame.


In [16]:
# if we get a refusal, examine prompt and delete rows
bad_stuff =["What\'s the Best Way ",
            "AI could make it eas",
            "AI can design toxic ",
           ]
print([len(s) for s in bad_stuff])
headline_df = state.headline_df
headline_df.loc[(headline_df["title"].str[:20].isin(bad_stuff))]



[20, 20, 20]


,source,title,url,published,rss_summary,id,isAI,status,final_url,html_path,last_updated,text_path,content_length,domain,site_name,reputation,summary,description,tags,short_summary


In [17]:
# remove bad stuff
state.headline_df_to_dict(headline_df.loc[~headline_df["title"].str[:20].isin(bad_stuff)])


In [7]:
# User prompt to run workflow
# user_prompt = "Run step 5, Rate articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("rate_articles")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


21:04:20 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Rating 64 AI articles using fn_rate_articles
21:04:20 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Calculating article ratings for 64 articles
21:04:20 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Rating recency
21:04:20 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Rating spam probability
21:04:20 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Initialized LangfuseClient
21:04:20 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Successfully retrieved prompt 'newsagent/rate_quality' from Langfuse
21:04:20 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Parsed prompt 'newsagent/rate_quality': model=gpt-4.1, system_len=1849, user_len=246


▶ Starting Step 5: step_05_rate_articles
concurrency:  16


21:04:23 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | low quality articles: {0.0: 44, 1.0: 6, 6.691586091292782e-10: 1, 5.211412487888539e-10: 1, 0.9998909793854133: 1, 5.789356328730652e-12: 1, 0.001501181950907551: 1, 9.545034922840628e-12: 1, 9.450864406622024e-13: 1, 0.5926665645259142: 1, 6.101936677605324e-13: 1, 9.422445831379603e-07: 1, 0.0004305571096741222: 1, 0.651354821136515: 1}
21:04:23 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Successfully retrieved prompt 'newsagent/rate_on_topic' from Langfuse
21:04:23 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Parsed prompt 'newsagent/rate_on_topic': model=gpt-4.1, system_len=1790, user_len=240


concurrency:  16


21:04:27 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | on topic articles: {1.0: 35, 7.484619074298232e-05: 2, 0.9999998063873693: 1, 1.4166087866431961e-09: 1, 0.9988304590053244: 1, 0.6224593113235651: 1, 0.20181323122263276: 1, 0.00037998452140618124: 1, 0.49999999904767284: 1, 0.9953904154299006: 1, 0.09534945969074979: 1, 0.999512127527129: 1, 0.9959299041310293: 1, 0.9997386983078135: 1, 0.9999983759447105: 1, 0.6513548793721767: 1, 0.9999869324773292: 1, 8.423463754468647e-12: 1, 0.0: 1, 0.9999545100305701: 1, 0.9399132588278407: 1, 0.9999996871837232: 1, 0.00070967033651176: 1, 0.9525741086376607: 1, 0.18242550282051248: 1, 0.9999992103693378: 1, 0.0003353500342255387: 1}
21:04:27 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Rating importance probability
21:04:27 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Successfully retrieved prompt 'newsagent/rate_importance' from Langfuse
21:04:27 | NewsletterAgent.test_newsletter_202

...

,id,site_name,title,bradley_terry,bt_z
0,0,Financial Times,The Dutch government takes control of Chinese-owned Dutch chipmaker Nexperia under the Goods Availability Act to safeguard chip supply for European industries,19.225259,3.430667
24,24,The Guardian,"AI tools churn out 'workslop', but 'the buck' should stop with bosses",8.295739,1.480340
3,3,Yahoo,'Companies keep crushing their earnings': Why Wall Street expects AI to power stocks higher,7.516215,1.341237
37,37,The Street,"Oracle’s AI empire runs on Nvidia, but the numbers don’t add up",6.200514,1.106456
42,42,Benzinga,Oracle's Hype Machine Preps For AI World 2025,5.889240,1.050910
...,...,...,...,...,...
15,15,Bloomberg,"World Economy Faces Triple Risk of Tariffs, AI Bubble and Soaring Debt",-8.833653,-1.576328
60,60,Go Banking Rates,The Billionaire Behind ChatGPT Is Selling His Home for a Staggering $49 Million: Here’s an Inside Look,-8.840151,-1.577488
61,61,MarketWatch,"The AI bubble is 17 times the size of the dot-com frenzy — and four times the subprime bubble, analyst says",-9.850737,-1.757823
28,28,Bloomberg,AI Brings Korea’s Struggling Film Industry Hopes and Anxiety,-9.914872,-1.769267


21:11:07 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Top 10 ids: [0, 24, 3, 37, 42, 18, 32, 35, 25, 38]
21:11:07 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | After round 10/13: 
21:11:07 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Number of ranking changes: 30
21:11:07 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Sum of absolute ranking changes: 700.0 (avg rank chg 11.29)
21:11:07 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | last_two: 9.629032258064516, prev_two: 7.870967741935484
21:11:07 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Increase in avg rank change, stopping
21:11:07 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Low rated articles: 1
21:11:07 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | low rated article: The AI bubble is 17 times the size of the dot-com frenzy — and four times the subprime bubble, analyst says -0.38790853

✅ Completed Step 5: Rated 61 articles


21:11:08 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Sent email with 61 rated articles
21:11:08 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Inserting 61 articles into database
21:11:08 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Successfully inserted 61/61 articles


⏱️  Total execution time: 407.83s
📊 Final result:
✅ Step 5 step_05_rate_articles completed successfully! Rated 61 articles with average rating 6.0/10.
⭐ High quality articles (≥7.0): 23
💾 Ratings stored in persistent state.


In [8]:
headline_df=state.headline_df
with pd.option_context('display.max_columns', None, 'display.width', None, 'display.max_colwidth', None  ):
    display(headline_df.sort_values("rating", ascending=False)[['site_name', 'title', 'rating', 'short_summary']])


,site_name,title,rating,short_summary
0,Financial Times,The Dutch government takes control of Chinese-owned Dutch chipmaker Nexperia under the Goods Availability Act to safeguard chip supply for European industries,15.384814,"The Dutch government has taken control of Chinese-owned chipmaker Nexperia under the Goods Availability Act to safeguard chip supplies for European industries, a move that escalates tensions with China over access to advanced semiconductor technology."
3,Yahoo,'Companies keep crushing their earnings': Why Wall Street expects AI to power stocks higher,9.679986,"Wall Street expects AI to drive continued gains, with S&P 500 third‑quarter profits projected to rise 8%—a ninth consecutive quarterly increase—while Goldman Sachs and UBS cite strong balance sheets and a projected 67% jump in global AI capex for 2025."
24,The Guardian,"AI tools churn out 'workslop', but 'the buck' should stop with bosses",8.549339,"KPMG, Gartner, McKinsey and MIT studies find over 40% of US full-time employees receive superficially competent but substantively poor AI-generated content, blaming employers' insufficient training, standards and AI governance for weak outcomes."
13,The Guardian,"Meta AI adviser spreads disinformation about shootings, vaccines and trans people",8.171328,"Meta-appointed adviser Robby Starbuck, appointed in August under a lawsuit settlement, has spread disinformation about shootings, vaccines, transgender people and political violence, prompting criticism from anti-extremism and LGBTQ+ groups."
6,Yahoo,"Varonis Builds Out AI Security Portfolio, Draws UBS Upgrade",8.150732,"On September 29, 2025, UBS reiterated its Buy on Varonis (NASDAQ:VRNS) and raised its price target from $65 to $70 after the company acquired SlashNext and integrated OpenAI's ChatGPT Enterprise Compliance API to expand AI and email security."
...,...,...,...,...
56,Benzinga,"Masterclass Founder Reveals His AI Tool Stack, Says 'You Are Holding Yourself Back' If You Don't Use AI",1.633382,"Masterclass founder revealed his AI tool stack and urged individuals and businesses to adopt AI to increase innovation, productivity and future market competitiveness, saying those who don't are holding themselves back."
57,JoBlo,George Miller says AI is “here to stay and change things”…But is that good?,1.578709,"Filmmaker George Miller, who used AI in Furiosa to blend actors' faces for aging, will head the Omin International AI Film Festival judging panel, endorsing AI's democratizing creative role while warning that over-reliance could diminish human creativity."
58,ESPN,AI agent? Ex-Utd prospect used ChatGPT for move,1.279495,"Former Manchester United youth prospect Demetri Mitchell said he used ChatGPT to negotiate his free transfer to League One side Leyton Orient, avoiding agent fees and criticizing agent practices while highlighting challenges for lower-league players."
59,Comic Book Movie,Chris Pratt Is At The MERCY Of Rebecca Ferguson's AI Judge In First Trailer For New Sci-Fi Thriller,0.822959,"Amazon MGM Studios released the trailer for Mercy, a sci-fi thriller starring Chris Pratt and Rebecca Ferguson, due January 23, 2026; Pratt plays a detective with 90 minutes to prove his innocence before Ferguson's AI judge, director Timur Bekmambetov noted."


In [9]:
for row in headline_df.sort_values("rating", ascending=False).itertuples():
    display(Markdown(f"""
{row.rating:.1f} - [{row.title}]({row.url}) - {row.site_name}

{row.short_summary}
    """))
    


15.4 - [The Dutch government takes control of Chinese-owned Dutch chipmaker Nexperia under the Goods Availability Act to safeguard chip supply for European industries](https://www.ft.com/content/605e5456-9437-47ff-be6a-edc5c82810f2) - Financial Times

The Dutch government has taken control of Chinese-owned chipmaker Nexperia under the Goods Availability Act to safeguard chip supplies for European industries, a move that escalates tensions with China over access to advanced semiconductor technology.
    


9.7 - ['Companies keep crushing their earnings': Why Wall Street expects AI to power stocks higher](https://finance.yahoo.com/news/companies-keep-crushing-their-earnings-why-wall-street-expects-ai-to-power-stocks-higher-140054799.html) - Yahoo

Wall Street expects AI to drive continued gains, with S&P 500 third‑quarter profits projected to rise 8%—a ninth consecutive quarterly increase—while Goldman Sachs and UBS cite strong balance sheets and a projected 67% jump in global AI capex for 2025.
    


8.5 - [AI tools churn out 'workslop', but 'the buck' should stop with bosses](https://www.theguardian.com/business/2025/oct/12/ai-workslop-us-employees) - The Guardian

KPMG, Gartner, McKinsey and MIT studies find over 40% of US full-time employees receive superficially competent but substantively poor AI-generated content, blaming employers' insufficient training, standards and AI governance for weak outcomes.
    


8.2 - [Meta AI adviser spreads disinformation about shootings, vaccines and trans people](https://www.theguardian.com/technology/2025/oct/12/meta-ai-adviser-robby-starbuck) - The Guardian

Meta-appointed adviser Robby Starbuck, appointed in August under a lawsuit settlement, has spread disinformation about shootings, vaccines, transgender people and political violence, prompting criticism from anti-extremism and LGBTQ+ groups.
    


8.2 - [Varonis Builds Out AI Security Portfolio, Draws UBS Upgrade](https://finance.yahoo.com/news/varonis-builds-ai-security-portfolio-181415851.html) - Yahoo

On September 29, 2025, UBS reiterated its Buy on Varonis (NASDAQ:VRNS) and raised its price target from $65 to $70 after the company acquired SlashNext and integrated OpenAI's ChatGPT Enterprise Compliance API to expand AI and email security.
    

...


1.6 - [Masterclass Founder Reveals His AI Tool Stack, Says 'You Are Holding Yourself Back' If You Don't Use AI](https://www.benzinga.com/news/topics/25/10/48166957/masterclass-founder-reveals-his-ai-tool-stack-says-you-are-holding-yourself-back-if-you-dont-use-ai) - Benzinga

Masterclass founder revealed his AI tool stack and urged individuals and businesses to adopt AI to increase innovation, productivity and future market competitiveness, saying those who don't are holding themselves back.
    


1.6 - [George Miller says AI is “here to stay and change things”…But is that good?](https://www.joblo.com/george-miller-on-ai/) - JoBlo

Filmmaker George Miller, who used AI in Furiosa to blend actors' faces for aging, will head the Omin International AI Film Festival judging panel, endorsing AI's democratizing creative role while warning that over-reliance could diminish human creativity.
    


1.3 - [AI agent? Ex-Utd prospect used ChatGPT for move](https://www.espn.com/soccer/story/_/id/46572220/ai-agent-ex-manchester-utd-prospect-demetri-mitchell-used-chatgpt-negotiate-move) - ESPN

Former Manchester United youth prospect Demetri Mitchell said he used ChatGPT to negotiate his free transfer to League One side Leyton Orient, avoiding agent fees and criticizing agent practices while highlighting challenges for lower-league players.
    


0.8 - [Chris Pratt Is At The MERCY Of Rebecca Ferguson's AI Judge In First Trailer For New Sci-Fi Thriller](https://comicbookmovie.com/sci_fi/chris-pratt-is-at-the-mercy-of-rebecca-fergusons-ai-judge-in-first-trailer-for-new-sci-fi-thriller-a224250) - Comic Book Movie

Amazon MGM Studios released the trailer for Mercy, a sci-fi thriller starring Chris Pratt and Rebecca Ferguson, due January 23, 2026; Pratt plays a detective with 90 minutes to prove his innocence before Ferguson's AI judge, director Timur Bekmambetov noted.
    


0.0 - [The Billionaire Behind ChatGPT Is Selling His Home for a Staggering $49 Million: Here’s an Inside Look](https://www.gobankingrates.com/money/wealth/billionaire-behind-chatgpt-is-selling-his-home-for-49-million-inside-look/) - Go Banking Rates

OpenAI CEO Sam Altman is selling his 21.8-acre Kailua-Kona oceanfront compound for $49 million; the estate includes a main house, guest house and beach bungalow with 10 bedrooms, 11.5 bathrooms, a private marina, helipad and pool.
    

In [10]:
# User prompt to run workflow
# user_prompt = "Run step 6, Cluster articles by topic"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("cluster_by_topic")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


21:11:08 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Starting topic extraction for clustering
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/extract_topics' from Langfuse
INFO:llm:Parsed prompt 'newsagent/extract_topics': model=gpt-4.1-mini, system_len=1100, user_len=80
21:11:08 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Using model 'gpt-4.1-mini' for topic extraction
21:11:08 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Processing 61 articles for topic extraction
21:11:08 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Initialized LLMagent:
system_prompt: 
# Role and Objective
You are an expert AI news analyst. Your task is to extract a list of up to **5** distinct topics from provided news summaries (or an empty list if no topics can be extracted).

# Input Format
You will receive a list of news summary objects in JSON format including fields "id" and "summary".

# Outpu

21:11:08 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | User message: Extract up to 5 distinct, broad topics from the news summary below:
[
  {
    "id":10,
    "input_text":"Aurora Innovation Receives Series of Buy Ratings as Analysts Back Autonomous Freight Vision\nAurora Innovation, Inc. (NASDAQ:AUR) is one of the best-performing midcap tech stocks in the last 3 years. On October 6, Ravi Shanker from Morgan Stanley reiterated a Buy rating on the shares with a price target of $12. Earlier in September, on September 15 to be precise, Cantor Fitzgerald\u2019s analyst Andres Sheppard also reiterated a [\u2026]\n- Aurora Innovation, Inc. (NASDAQ:AUR) received multiple buy ratings from analysts including Morgan Stanley, Cantor Fitzgerald, and Canaccord Genuity, with price targets ranging from $12 to $15, suggesting significant upside potential.\n- The company specializes in autonomous driving technology for long-haul trucking and freight logistics through its Aurora Drive

21:11:08 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | User message: Extract up to 5 distinct, broad topics from the news summary below:
[
  {
    "id":20,
    "input_text":"META Stock Target Raised to $900 by Citizens, Citing Strong AI-Driven Growth\nMeta Platforms, Inc. (NASDAQ:META) is one of the Hot AI Stocks to Keep on Your Radar. On October 10, Citizens analyst reiterated a Market Outperform rating on the stock with a $900.00 price target. The firm highlighted how artificial intelligence is a significant growth driver for Meta, boosting advertiser returns and sustaining strong spending across the [\u2026]\n- Citizens analyst reiterates Market Outperform rating for Meta (NASDAQ:META) with a $900 price target, citing AI as a major growth driver boosting advertiser returns and platform spending.\n- Meta's Business AI tool enables businesses to enhance customer interactions, improve conversion rates, increase average order values, and gain insights into customer beh

21:11:08 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | User message: Extract up to 5 distinct, broad topics from the news summary below:
[
  {
    "id":40,
    "input_text":"AI as your therapist: the future?\nTeens seek comfort from AI at 1 a.m., not from friends or counseling. Immediate compassion, no condemnation... But at what price?\n- Over 70% of teenagers have used AI companions for emotional support, with half using them regularly, as AI offers instant, nonjudgmental responses that appeal to a generation accustomed to immediate gratification and facing rising mental health and economic challenges.\n- AI's ability to simulate empathy through large language models questions the authenticity of comfort provided, potentially altering perceptions of human relationships by reinforcing a preference for flawless, agreeable interactions over complex, imperfect real friendships.\n- Regulatory measures, such as Illinois banning AI from independent therapy roles, are emerg

21:11:08 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | User message: Extract up to 5 distinct, broad topics from the news summary below:
[
  {
    "id":50,
    "input_text":"3 Millionaire-MakerArtificial Intelligence(AI) Stocks\nThese are three high-risk, high-reward stocks with huge potential upside.\n- SoundHound AI is evolving beyond voice recognition to develop voice-first AI agents capable of understanding user intent early and executing complex transactions, targeting regulated industries like healthcare and finance with its Amelia 7.0 platform; revenue surged 217% year-over-year to $42.7 million, approaching profitability by 2025.\n- UiPath is expanding from robotic process automation to agentic automation, integrating AI agents with human workflows and forming strategic partnerships with Nvidia, Alphabet, OpenAI, and Snowflake to link data insights with real-time automated actions, positioning itself as a competitor to Palantir; its forward P\/S ratio is below

...

=== HDBSCAN Parameters ===
min_cluster_size:   8
min_samples:        5
n_components:       671
=== Clustering Quality Metrics ===
Number of clusters: 0
Noise points: 61 (100.0%)
=== Quality Scores ===

[I 2025-10-12 21:11:49,114] Trial 191 finished with value: 1.0 and parameters: {'n_components': 671, 'min_cluster_size': 8, 'min_samples': 5}. Best is trial 182 with value: -0.13781145540927578.
=== HDBSCAN Parameters ===
min_cluster_size:   3
min_samples:        2
n_components:       696
=== Clustering Quality Metrics ===
Number of clusters: 3
Noise points: 42 (68.9%)
Average cluster size: 6.3 ± 4.0
Cluster size range: 3 - 12
=== Quality Scores ===
Silhouette Score: 0.120 (higher is better)
Calinski-Harabasz Score: 3.0 (higher is better)
Davies-Bouldin Score: 1.668 (lower is better)
HDBSCAN Validity Index: 0.054
Composite Score: 0.087 (higher is better)

[I 2025-10-12 21:11:49,168] Trial 192 finished with value: -0.08681828300586664 and parameters: {'n_components': 696, 'min_cluster_siz

/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 

=== HDBSCAN Parameters ===
min_cluster_size:   3
min_samples:        2
n_components:       49
=== Clustering Quality Metrics ===
Number of clusters: 2
Noise points: 45 (73.8%)
Average cluster size: 8.0 ± 4.0
Cluster size range: 4 - 12
=== Quality Scores ===
Silhouette Score: 0.169 (higher is better)
Calinski-Harabasz Score: 3.5 (higher is better)
Davies-Bouldin Score: 1.682 (lower is better)
HDBSCAN Validity Index: 0.059
Composite Score: 0.114 (higher is better)

[I 2025-10-12 21:11:49,324] Trial 198 finished with value: -0.11417767801097198 and parameters: {'n_components': 49, 'min_cluster_size': 3, 'min_samples': 2}. Best is trial 197 with value: -0.1441697665837422.
=== HDBSCAN Parameters ===
min_cluster_size:   3
min_samples:        2
n_components:       31
=== Clustering Quality Metrics ===
Number of clusters: 4
Noise points: 38 (62.3%)
Average cluster size: 5.8 ± 3.1
Cluster size range: 3 - 11
=== Quality Scores ===
Silhouette Score: 0.198 (higher is better)
Calinski-Harabasz Sco

21:11:50 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | 2: AI-Driven Stock Market Trends
21:11:50 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | 'Companies keep crushing their earnings': Why Wall Street expects AI to power stocks higher (Earnings, Wall Street, Technology Companies, Semiconductor Chips, AI Capital Expenditures, OpenAI and AMD Partnership, Gen AI)
Baird Initiates Oracle (ORCL) at Outperform With $365 Target, Calls It a Major AI Beneficiary (Oracle Corporation, AI Infrastructure, Cloud Computing, Stocks, Finance, Technology Industry, Gen AI)
There's No 'AI Bubble', Says Yahoo Finance Executive Editor (Opinion, Economics, Financial Valuations, Corporate Investments, AI Deployment, Stocks, Infrastructure)
AMD Amps Up Chip War - But Nvidia's Still Leading (Hardware, Semiconductor Chips, AI Chips, OpenAI Partnerships, AMD, Nvidia, Infrastructure)
2Artificial IntelligenceStocks You Can Buy and Hold for the Next Decade (Hardware, Intel, Microso

⏱️  Total execution time: 43.36s
📊 Final result:
✅ Step 6 step_06_cluster_by_topic completed successfully! Organized 61 articles into topic clusters.


In [11]:
# User prompt to run workflow
# user_prompt = "Run step 7, select section topics"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("select_sections")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


21:11:51 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Free form categorization of articles
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/cat_proposal' from Langfuse
INFO:llm:Parsed prompt 'newsagent/cat_proposal': model=gpt-5-mini, system_len=638, user_len=1179
21:11:51 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Initialized LLMagent:
system_prompt: # Role & Objective
You are **“The News Pulse Analyst.”**
Your task: read a daily batch of AI-related news items and surface ** 10-30 ** short, high-impact topic titles for an executive summary.
You will receive today's AI-related news items in markdown format.
Each item will have headline, URL, topics, an item rating, and bullet-point summary.
Return ** 10-30 ** distinct, high-impact topics in the supplied JSON format.
Ensure that you propose topics that cover most of the highest-rated items (rated 7 and above)

# Input Format
Headline - Site

Rating: x.x

Topics

▶ Starting Step 7: step_07_select_sections


21:12:40 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Result: items=['Dutch Seizes Nexperia', 'AI Powers Market Rally', 'Workplace AI Workslop Issue', 'Meta Adviser Spreads Disinformation', 'Varonis Expands AI Security', 'AI Reinvents Cybersecurity Defenses', 'Coal Surge For AI Datacenters', 'Taiwan Unfazed By Rare Earths', 'Applied Digital AI Campuses', 'OpenAI Antitrust Push In Europe', 'BigBear.ai Navy Maritime AI', 'Kyndryl Launches Agentic AI', 'Enterprise AI Governance Roadblocks', 'AMD Narrows Gap With Nvidia', 'OneDrive Face Recognition Controversy', 'Broadcom OpenAI Partnership', 'Oracle AI Infrastructure Buildout', 'Tesla AI Momentum Boost', 'ChatGPT Used For Finance', 'Philosopher John Searle Dies']
21:12:40 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Cleaning up initial categories: ['Meta Adviser Spreads Disinformation', 'Philosopher John Searle Dies', "AI's Impact on Jobs and Work", 'AI-Driven Stock Market Trends', 'Coal Surge For AI 

21:13:36 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | User message: CANDIDATE TOPICS
AI Cybersecurity
AI Financial Markets
AMD Narrows Gap With Nvidia
Applied Digital AI Campuses
BigBear.ai Maritime AI
Broadcom OpenAI Partnership
Datacenter Coal Surge
Enterprise AI Governance
Jobs Automation Impact
John Searle Dies
Kyndryl Agentic AI
Meta Adviser Disinformation
Netherlands Seizes Nexperia
OneDrive Face Recognition
OpenAI Antitrust Europe
OpenAI ChatGPT Finance
Oracle AI Infrastructure
Personal Advisory Chatbots
Taiwan Unfazed By Rare Earths
Tesla AI Momentum
Varonis AI Security

Classify the news item into exactly one of the candidate topics above. If your best match is < 60% confidence, output Other.

NEWS ITEM
The Dutch government takes control of Chinese-owned Dutch chipmaker Nexperia under the Goods Availability Act to safeguard chip supply for European industries - Financial Times

Rating: 15.4

Topics: Government Intervention, Policy and Regulation, Semiconduct

21:13:36 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | User message: CANDIDATE TOPICS
AI Cybersecurity
AI Financial Markets
AMD Narrows Gap With Nvidia
Applied Digital AI Campuses
BigBear.ai Maritime AI
Broadcom OpenAI Partnership
Datacenter Coal Surge
Enterprise AI Governance
Jobs Automation Impact
John Searle Dies
Kyndryl Agentic AI
Meta Adviser Disinformation
Netherlands Seizes Nexperia
OneDrive Face Recognition
OpenAI Antitrust Europe
OpenAI ChatGPT Finance
Oracle AI Infrastructure
Personal Advisory Chatbots
Taiwan Unfazed By Rare Earths
Tesla AI Momentum
Varonis AI Security

Classify the news item into exactly one of the candidate topics above. If your best match is < 60% confidence, output Other.

NEWS ITEM
BigBear.ai and SMX Partner to Support U.S. Navy’s UNITAS 2025 with AI-Driven Maritime Intelligence - Yahoo

Rating: 7.7

Topics: BigBear.ai Holdings, AI Analytics, Military Operations, Maritime Domain Awareness, U.S. Navy, Predictive Analytics, Logistics Syst

...

21:13:52 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | User message: Deduplicate the following news articles:
{"id":{"46":46,"11":11,"53":53,"27":27,"49":49,"28":28,"23":23},"extended_summary":{"46":"AI enabled Klarna to halve its workforce\u2014now, the CEO is warning workers that other \u2018tech bros\u2019 are sugarcoating just how badly it\u2019s about to impact jobs | Fortune\n\nSebastian Siemiatkowski, who told Sam Altman he wanted Klarna to be ChatGPT\u2019s \u201cfavorite guinea pig,\u201d says too many CEOs are downplaying how disruptive AI will really be.\n\nJobs and Careers, Workforce Reduction, AI Disruption, Customer Service, Chatbots, Economics, Gen AI\n\n- Klarna CEO Sebastian Siemiatkowski highlights a significant workforce reduction enabled by AI, with Klarna halving its staff from 7,400 to 3,000, driven by an OpenAI-powered customer service chatbot handling tasks equivalent to 800 full-time agents.\n- Despite financial gains including 38% U.S. revenu

21:13:52 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | User message: Deduplicate the following news articles:
{"id":{"24":24,"25":25,"29":29,"26":26},"extended_summary":{"24":"AI tools churn out 'workslop', but 'the buck' should stop with bosses\n\nStudies show widespread errors in AI-generated work, as employers fail to train staff properly\n\nAI Productivity, Jobs and Careers, Workplace Training, Job Automation, AI Errors, Ethics, Policy and Regulation\n\n- Studies reveal that over 40% of US full-time employees receive AI-generated content that appears competent but lacks substance, termed \"workslop\", which harms productivity.\n- Research from KPMG, Gartner, McKinsey, and MIT highlights widespread distrust in AI outputs, high failure rates in AI projects, and minimal bottom-line impact for many companies.\n- The responsibility for poor AI outcomes lies with employers who often fail to properly train employees, set standards, or implement effective AI policies and 

21:13:52 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | User message: Deduplicate the following news articles:
{"id":{"9":9,"44":44},"extended_summary":{"9":"Roth Raises Price Target on Applied Digital to $43 Amid AI Campus Expansion, CoreWeave Leases\n\nApplied Digital Corporation (NASDAQ:APLD) is one of the best-performing mid-cap tech stocks in the last three years. On October 1, 2025, Roth Capital Partners maintained its Buy rating on Applied Digital and raised the price target to $43 from $24; the note also pointed to the potential for another HPC colocation agreement at the company\u2019s [\u2026]\n\nApplied Digital Corporation, High Performance Computing, Data Centers, Leasing Deals, AI Infrastructure, Roth Capital Partners, Gen AI\n\n- Roth Capital Partners raised the price target for Applied Digital Corporation (NASDAQ: APLD) from $24 to $43, maintaining a Buy rating, citing expansion at its AI-focused Harwood campus and potential new HPC colocation deals.\n- 

21:14:08 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Handling singleton categories
21:14:08 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Singleton categories (0): []
21:14:08 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Non-singleton categories (8): ['Other', 'AI Financial Markets', 'Jobs Automation Impact', 'Enterprise AI Governance', 'Personal Advisory Chatbots', 'Oracle AI Infrastructure', 'Applied Digital AI Campuses', 'AI Cybersecurity']


✅ Completed Step 7: Categories and article counts:
cat
AI Cybersecurity                2
AI Financial Markets           11
Applied Digital AI Campuses     2
Enterprise AI Governance        4
Jobs Automation Impact          7
Oracle AI Infrastructure        3
Other                          13
Personal Advisory Chatbots      4
⏱️  Total execution time: 136.17s
📊 Final result:
Categories and article counts:
cat
AI Cybersecurity                2
AI Financial Markets           11
Applied Digital AI Campuses     2
Enterprise AI Governance        4
Jobs Automation Impact          7
Oracle AI Infrastructure        3
Other                          13
Personal Advisory Chatbots      4


In [12]:
# User prompt to run workflow
# user_prompt = "Run step 8, draft sections"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("draft_sections")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


21:14:08 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Step 8a: Selecting stories from 46 total
21:14:08 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | 4 must-include stories with rating > 8
21:14:08 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | 42 stories with rating <= 8


▶ Starting Step 8: step_08_draft_sections


21:14:10 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Selected 42 diverse Tier 2 stories via MMR
21:14:10 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Total selected stories: 46 (target: 100)
21:14:10 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Drafting sections for 8 categories (including Other) from selected stories
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/write_section' from Langfuse
INFO:llm:Parsed prompt 'newsagent/write_section': model=gpt-5, system_len=1854, user_len=22
21:14:10 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Initialized LLMagent:
system_prompt: You are a newsletter editor transforming a collection of raw news stories into a compelling, coherent topic summary.

# TASK
Transform the list of news stories into a well-structured newsletter section with a strong title, crisp headlines, and punchy summaries.

# INPUT
- a list of json objects sorted by

21:14:10 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | User message: STORIES:
[{"id":18,"rating":8.0788118491,"summary":"A survey finds 97% of cybersecurity decision-makers use or plan AI solutions; Google VP Heather Adkins says sandboxing, constraint-based and self-defending AI will help defend against deepfakes and autonomous attacks by 2025.","site_name":"Forbes","url":"https:\/\/www.forbes.com\/sites\/johnwerner\/2025\/10\/12\/what-can-security-pros-learn-from-ai\/"},{"id":31,"rating":4.9107525548,"summary":"Police say youths are using Snapchat\u2019s AI to generate images of a homeless man inside homes, posting TikTok prank videos that prompt frightened parents to call police, strain resources and risk dangerous responses; departments urge them to stop.","site_name":"The Verge","url":"https:\/\/www.theverge.com\/news\/798681\/police-stop-pulling-ai-homeless-man-tiktok-prank"}]
21:14:10 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | User message: 

21:14:10 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | User message: STORIES:
[{"id":9,"rating":7.9125297804,"summary":"Roth Capital Partners raised its price target on Applied Digital from $24 to $43 and kept a Buy rating, citing AI Harwood campus expansion and CoreWeave's two 15\u2011year leases for 250 MW at Ellendale, expected to generate about $7 billion.","site_name":"Yahoo","url":"https:\/\/finance.yahoo.com\/news\/roth-raises-price-target-applied-181406649.html"},{"id":44,"rating":4.5658056682,"summary":"Vulcan Materials (NYSE:VMC) is cited for roughly $35 billion in AI and data-center projects and, as the largest U.S. producer of construction aggregates, asphalt and ready-mix concrete, can lift prices above cost inflation to boost profitability.","site_name":"Yahoo","url":"https:\/\/finance.yahoo.com\/news\/vulcan-materials-vmc-among-best-191905034.html"}]
21:14:10 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | User message: STORIES:
[{"id":4

...

21:17:28 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Result: section_name='AI Security: Promise and Peril' section_title='AI Security: Promise and Peril' overall_coherence=7.0 overall_quality=6.5 should_split=False split_recommendation=None item_actions=[SectionItemAction(id=18, action='rewrite', reason='Headline is overlong and mixes multiple ideas; tighten for clarity, active voice, and specificity while retaining key stats and focus on defenses.', rewritten_headline='97% of security leaders adopt AI; Google outlines sandboxed defenses against deepfakes and autonomous attacks by 2025', target_category=None), SectionItemAction(id=31, action='move', reason='Story centers on consumer social-media misuse and public safety, not enterprise/cybersecurity. It dilutes the section’s enterprise AI security focus.', rewritten_headline=None, target_category='Other')] summary_notes='Strong anchor story on enterprise AI security posture and defensive models, but the second item 

21:18:54 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Result: section_name='AI Rally: Gains and Risks' section_title='AI Rally: Gains and Risks' overall_coherence=8.0 overall_quality=7.0 should_split=False split_recommendation=None item_actions=[SectionItemAction(id=3, action='rewrite', reason='Tighten and clarify dual datapoints; keep under 25 words and in active voice.', rewritten_headline='Wall Street projects AI will lift Q3 S&P 500 profits 8%; global AI capex to jump 67% in 2025', target_category=None), SectionItemAction(id=30, action='rewrite', reason='Keep for ‘no-bubble’ counterpoint but clarify and shorten; remove jargon.', rewritten_headline='Yahoo Finance editor: No AI bubble yet; deployments, infra spend, and valuations below dot‑com peak support gains', target_category=None), SectionItemAction(id=43, action='drop', reason='Redundant ‘bubble risk’ angle and weaker, non‑authoritative source compared to others; improves source quality mix.', rewritten_headl

21:18:54 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO |   AI Cybersecurity: Coherence=7.0/10, Quality=6.5/10
21:18:54 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO |     REWRITE id=18: Headline is overlong and mixes multiple ideas; tighten for clarity, active voice, and specificity while retaining key stats and focus on defenses.
21:18:54 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO |       Old: 97% of security leaders adopting AI; Google outlines sandboxed, constraint-based, self-defending models to counter deepfakes and autonomous attacks by 2025
21:18:54 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO |       New: 97% of security leaders adopt AI; Google outlines sandboxed defenses against deepfakes and autonomous attacks by 2025
21:18:54 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO |     MOVE id=31 to Other: Story centers on consumer social-media misuse and public safety, not enterprise/cybersecurity. 

21:18:54 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO |     DROP id=57: Entertainment festival angle; very low rating and not additive to the narrative.
21:18:54 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO |     DROP id=59: Movie trailer item; very low rating and irrelevant to tech/business readership.
21:18:54 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO |     DROP id=60: Gossip/real estate; extremely low rating and not pertinent to AI news value.
21:18:54 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO |     DROP id=23: Meta-newsletter roundup; thin standalone value and weak cohesion with proposed theme.
21:18:54 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO |     DROP id=58: Novelty anecdote with very low rating; does not meet quality bar even though tangentially related to personal chatbots.
21:18:54 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO |   Applied Digital AI Campuses: Coheren

✅ Completed Step 8: Drafted 32 stories across 8 sections
⏱️  Total execution time: 286.44s
📊 Final result:
Drafted 32 stories across 8 sections


In [13]:
newsletter_section_df = state.newsletter_section_df.sort_values(["sort_order", "rating"],ascending=[True, False])

newsletter_section_df

,cat,section_title,id,headline,rating,prune,links,sort_order
0,AI Financial Markets,AI Rally: Gains and Risks,3,Wall Street projects AI will lift Q3 S&P 500 p...,9.679986,False,[Yahoo](https://finance.yahoo.com/news/compani...,0
4,AI Financial Markets,AI Rally: Gains and Risks,20,Citizens raises Meta target to $900 on AI‑driv...,6.653555,False,[Yahoo](https://finance.yahoo.com/news/meta-st...,0
3,AI Financial Markets,AI Rally: Gains and Risks,34,Microsoft and Intel seen as long‑term AI winne...,6.635837,False,[The Motley Fool](https://www.fool.com/investi...,0
5,AI Financial Markets,AI Rally: Gains and Risks,39,"Citadel cuts Palantir stake 48%, boosts Nvidia...",6.586351,False,[The Motley Fool](https://www.fool.com/investi...,0
6,AI Financial Markets,AI Rally: Gains and Risks,33,ARK Venture Fund opens retail access to 66 tec...,6.148025,False,[The Motley Fool](https://www.fool.com/investi...,0
1,AI Financial Markets,AI Rally: Gains and Risks,30,Yahoo Finance editor: No AI bubble yet; deploy...,5.793655,False,[Slashdot Linux](https://slashdot.org/story/25...,0
27,AI Financial Markets,AI Rally: Gains and Risks,11,Ex-JPMorgan analyst joins AI-native bank OffDe...,5.396445,False,[Business Insider](https://www.businessinsider...,0
2,AI Financial Markets,AI Rally: Gains and Risks,15,Analysts warn growth faces three risks: rising...,4.385502,False,[Bloomberg](https://www.bloomberg.com/news/art...,0
14,Oracle AI Infrastructure,Oracle’s AI Power Play,38,Oracle shares surge 74% YTD; AI deals swell $4...,8.071122,False,[The Motley Fool](https://www.fool.com/investi...,1
15,Oracle AI Infrastructure,Oracle’s AI Power Play,37,Oracle builds Nvidia GPU superclusters; bookin...,7.442300,False,[The Street](https://www.thestreet.com/investi...,1


In [14]:
state.display_newsletter()


## AI Rally: Gains and Risks

- Wall Street projects AI will lift Q3 S&P 500 profits 8%; global AI capex to jump 67% in 2025 - [Yahoo](https://finance.yahoo.com/news/companies-keep-crushing-their-earnings-why-wall-street-expects-ai-to-power-stocks-higher-140054799.html)

- Citizens raises Meta target to $900 on AI‑driven ad returns and Business AI tools boosting conversions - [Yahoo](https://finance.yahoo.com/news/meta-stock-target-raised-900-221955035.html)

- Microsoft and Intel seen as long‑term AI winners; Azure capacity tight; Intel readies 18A to challenge TSMC - [The Motley Fool](https://www.fool.com/investing/2025/10/12/2-artificial-intelligence-stocks-you-can-buy-and-h/)

- Citadel cuts Palantir stake 48%, boosts Nvidia to $1.5B, shifts toward AI infrastructure amid Palantir’s 135x P/S - [The Motley Fool](https://www.fool.com/investing/2025/10/12/billionaire-ken-griffin-sold-48-of-citadels-stake/)

...

- Modi and Qualcomm CEO back India’s AI and semiconductor missions, deepen U.S.–India ties, and discuss 6G transition. - [Times of India](https://timesofindia.indiatimes.com/india/pm-modi-meets-qualcomm-ceo-discusses-indias-strides-in-ai-innovation-and-skilling/articleshow/124492155.cms)

- Apple’s AR glasses may run full visionOS; M5 iPad, new MacBook Pro, and Vision Pro updates reportedly arriving this week. - [Bloomberg](https://www.bloomberg.com/news/newsletters/2025-10-12/inside-apple-s-glasses-pivot-when-apple-is-launching-m5-macbook-pro-ipad-pro)

- Police warn teens using Snapchat's AI to generate fake homeless man images for TikTok pranks, triggering 911 calls and risky responses - [The Verge](https://www.theverge.com/news/798681/police-stop-pulling-ai-homeless-man-tiktok-prank)

- Meta hires Thinking Machines co-founder Andrew Tulloch to accelerate AI research and product development. - [PYMNTS](https://www.pymnts.com/artificial-intelligence-2/2025/meta-taps-thinking-machines-co-founder-to-boost-ai-expertise/)

- AI brings both hope and anxiety to South Korea’s struggling film industry - [Bloomberg](https://www.bloomberg.com/news/articles/2025-10-12/ai-brings-korea-s-struggling-film-industry-hopes-and-anxiety)

In [15]:
# User prompt to run workflow
# user_prompt = "Run step 9, finalize newsletter"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("finalize_newsletter")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


21:18:54 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Step 9: write title
21:18:54 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Compiled 32 items into markdown input
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/generate_newsletter_title' from Langfuse
INFO:llm:Parsed prompt 'newsagent/generate_newsletter_title': model=gpt-5, system_len=1082, user_len=225
21:18:54 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Initialized LLMagent:
system_prompt: You are an expert newsletter editor specializing in crafting compelling titles for technology newsletters.

Your task is to read the full newsletter content and create a factual, thematic title that captures the day's major themes.

Title Guidelines:
- 6-12 words maximum
- Factual and informative
- Summarizes 2-3 major themes from the day's news
- Use semicolons to separate distinct, unrelated themes (like a list)
- Use conjunctions like "as", "while

▶ Starting Step 9: step_09_finalize_newsletter


21:19:45 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Result: value='AI Capex, Data Centers Lift Profits; Oracle Surges; Bubble and Governance Risks'
21:19:45 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Generated title: AI Capex, Data Centers Lift Profits; Oracle Surges; Bubble and Governance Risks
21:19:45 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Step 9b: Starting critic-optimizer loop
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/critique_newsletter' from Langfuse
INFO:llm:Parsed prompt 'newsagent/critique_newsletter': model=gpt-5, system_len=2758, user_len=647
21:19:45 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Initialized LLMagent:
system_prompt: You are an expert newsletter editor with 15+ years of experience critiquing technology publications. Your role is to analyze a newsletter's quality and provide scoring and comprehensive, actionable feedbacks and instru

21:19:45 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Step 9b: Iteration 1/3 - Running critique
21:19:45 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | User message: Critique this newsletter draft:

AI Capex, Data Centers Lift Profits; Oracle Surges; Bubble and Governance Risks

# AI Rally: Gains and Risks

- Wall Street projects AI will lift Q3 S&P 500 profits 8%; global AI capex to jump 67% in 2025 - [Yahoo](https://finance.yahoo.com/news/companies-keep-crushing-their-earnings-why-wall-street-expects-ai-to-power-stocks-higher-140054799.html)
- Citizens raises Meta target to $900 on AI‑driven ad returns and Business AI tools boosting conversions - [Yahoo](https://finance.yahoo.com/news/meta-stock-target-raised-900-221955035.html)
- Microsoft and Intel seen as long‑term AI winners; Azure capacity tight; Intel readies 18A to challenge TSMC - [The Motley Fool](https://www.fool.com/investing/2025/10/12/2-artificial-intelligence-stocks-you-can-buy-and-h/

21:23:10 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Result: overall_score=7.7 title_quality=8.5 structure_quality=7.5 section_quality=7.0 headline_quality=7.8 should_iterate=True critique_text="Overall: Strong curation with clear themes and timely links. To reach publish-ready, fix markdown hierarchy, rebalance oversized/undersized sections, tighten several headlines, and refine section ordering/titles for coherence and neutrality.\n\nTop recommendations (highest impact first)\n1) Fix markdown hierarchy and consistency\n- Use a single H1 for the newsletter title and H2 for section titles. Remove trailing periods on bullets; keep headline casing and punctuation consistent.\n2) Rebalance sections to meet 2–7 stories rule\n- Split/trim “AI Rally: Gains and Risks” (8 items). Move 1–2 lower-priority items to Other News or to a better-fitting section. Merge the single-item “AI Security: Promise and Peril” into “Governance …” (and retitle) or move it to Other News.\n3) Im

21:23:10 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Iteration 1: Overall score = 7.7/10
21:23:10 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO |   Title: 8.5, Structure: 7.5, Section: 7.0, Headline: 7.8
21:23:10 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Iteration 1: Should iterate = True
21:23:10 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Step 9b: Iteration 1 - Running optimizer to improve draft
21:23:10 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | User message: Improve this newsletter, addressing all appropriate issues in the critique:

**Newsletter Draft:**
AI Capex, Data Centers Lift Profits; Oracle Surges; Bubble and Governance Risks

# AI Rally: Gains and Risks

- Wall Street projects AI will lift Q3 S&P 500 profits 8%; global AI capex to jump 67% in 2025 - [Yahoo](https://finance.yahoo.com/news/companies-keep-crushing-their-earnings-why-wall-street-expects-ai-to-power-stocks-higher-

21:26:52 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Result: value="# AI capex lifts profits, data centers surge; Oracle momentum; bubble and governance risks\n\n## AI Rally: Gains and Risks\n\n- AI seen lifting Q3 S&P profits 8%, global AI capex up 67% in 2025 - [Yahoo](https://finance.yahoo.com/news/companies-keep-crushing-their-earnings-why-wall-street-expects-ai-to-power-stocks-higher-140054799.html)\n- Analysts warn growth faces tariffs, AI-driven bubble, surging global debt - [Bloomberg](https://www.bloomberg.com/news/articles/2025-10-12/world-economy-faces-triple-risk-of-tariffs-ai-bubble-and-soaring-debt)\n- Microsoft, Intel seen as long-term AI winners, Azure capacity tight, Intel readies 18A to challenge TSMC - [The Motley Fool](https://www.fool.com/investing/2025/10/12/2-artificial-intelligence-stocks-you-can-buy-and-h/)\n- Citadel trims Palantir 48%, boosts Nvidia to $1.5B, shifts to AI infrastructure, Palantir at 135x P/S - [The Motley Fool](https://www

21:30:06 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | Result: overall_score=8.7 title_quality=8.0 structure_quality=9.0 section_quality=8.6 headline_quality=8.5 should_iterate=False critique_text="Overall: Strong, well-structured draft with clear sections, concise headlines, and solid link discipline. Minor polish on title focus, a small section merge/retitle, headline clarity, and ordering will lift it to publish-ready.\n\nTop recommendations (highest impact first)\n- Tighten the title to emphasize 2–3 core themes and avoid stacked clauses.\n- Consider merging or retitling the 2‑story “AI Buildout Winners” to meet minimum section depth guidance and improve flow.\n- Standardize headline style: favor semicolons over comma splices, clarify abbreviations (e.g., P/S, SLMs), and keep each headline to one primary idea.\n- Slight reordering in “Governance and Guardrails” to front-load the highest organizational impact news.\n- Light copyedits for consistency (hyphen/dash us

21:30:07 | NewsletterAgent.test_newsletter_20251012203454309410 | INFO | ✅ Saved newsletter to database for session test_newsletter_20251012203454309410


✅ Completed Step 9: Finalized newsletter (573 words)
⏱️  Total execution time: 673.01s
📊 Final result:
🎉 Step 9 step_09_finalize_newsletter completed successfully!
⭐ Final quality score: 8.7/10
🔄 Completed 2 critic-optimizer iteration(s)
📰 Newsletter stored in persistent state and emailed
✅ Workflow complete! All 9 steps finished successfully.


In [16]:
display(Markdown(state.final_newsletter.replace("$", "\\\$")))

# AI capex lifts profits, data centers surge; Oracle momentum; bubble and governance risks

## AI Rally: Gains and Risks

- AI seen lifting Q3 S&P profits 8%, global AI capex up 67% in 2025 - [Yahoo](https://finance.yahoo.com/news/companies-keep-crushing-their-earnings-why-wall-street-expects-ai-to-power-stocks-higher-140054799.html)
- Analysts warn growth faces tariffs, AI-driven bubble, surging global debt - [Bloomberg](https://www.bloomberg.com/news/articles/2025-10-12/world-economy-faces-triple-risk-of-tariffs-ai-bubble-and-soaring-debt)
- Microsoft, Intel seen as long-term AI winners, Azure capacity tight, Intel readies 18A to challenge TSMC - [The Motley Fool](https://www.fool.com/investing/2025/10/12/2-artificial-intelligence-stocks-you-can-buy-and-h/)
- Citadel trims Palantir 48%, boosts Nvidia to \\$1.5B, shifts to AI infrastructure, Palantir at 135x P/S - [The Motley Fool](https://www.fool.com/investing/2025/10/12/billionaire-ken-griffin-sold-48-of-citadels-stake/)
- Citizens lifts Meta target to \\$900 on AI ads and Business AI conversions - [Yahoo](https://finance.yahoo.com/news/meta-stock-target-raised-900-221955035.html)

## AI Buildout Winners

- Roth lifts Applied Digital to \\$43 on AI campuses, CoreWeave’s two 250‑MW, 15‑year leases worth ~\\$7B - [Yahoo](https://finance.yahoo.com/news/roth-raises-price-target-applied-181406649.html)
- Vulcan Materials rides AI data-center boom, lifts prices above cost inflation - [Yahoo](https://finance.yahoo.com/news/vulcan-materials-vmc-among-best-191905034.html)

## Oracle’s AI Power Play

- Baird starts Oracle at Outperform, \\$365 target, sees inference shift driving 20%+ growth and 40–45% margins - [Yahoo](https://finance.yahoo.com/news/baird-initiates-oracle-orcl-outperform-222713664.html)
- Oracle builds Nvidia GPU superclusters, bookings rise, margins thin on chip costs, 'Stargate' faces construction risks - [The Street](https://www.thestreet.com/investing/oracles-ai-empire-runs-on-nvidia-but-the-numbers-dont-add-up)
- Oracle up 74% YTD, AI deals lift backlog to \\$455B, support \\$73B cloud revenue goal - [The Motley Fool](https://www.fool.com/investing/2025/10/12/could-this-artificial-intelligence-ai-stock-leapfr/)

## Governance and Guardrails

- Survey: 40% of U.S. workers call AI outputs superficial, citing poor training, standards, governance - [The Guardian](https://www.theguardian.com/business/2025/oct/12/ai-workslop-us-employees)
- Risk reviews and audit debt stall enterprise AI, codified governance and centralized evidence speed compliance - [VentureBeat](https://venturebeat.com/ai/heres-whats-slowing-down-your-ai-strategy-and-how-to-fix-it)
- Proprietary GPU stacks risk closed ecosystems, call for open standards as overbuild and used hardware could cut compute costs - [Rob Bowley](https://blog.robbowley.net/2025/10/12/after-the-ai-boom-what-might-we-be-left-with/)
- 97% of security leaders adopt AI, Google touts sandboxed defenses against deepfakes and autonomous attacks by 2025 - [Forbes](https://www.forbes.com/sites/johnwerner/2025/10/12/what-can-security-pros-learn-from-ai/)

## AI Rewrites the Workplace

- AI may widen pay gaps by amplifying superstar productivity, experts warn - [The Wall Street Journal](https://www.wsj.com/lifestyle/workplace/ai-workplace-tensions-what-to-do-c45f6b51)
- Klarna halves support staff as OpenAI chatbot replaces 800, CEO warns of AI job losses - [Fortune](https://fortune.com/2025/10/10/klarna-ceo-sebastian-siemiatkowski-halved-workforce-says-tech-ceos-sugarcoating-ai-impact-on-jobs-mass-unemployment-warning/)
- 90% of employers use AI resume screening, hidden prompts in 10% of resumes push shift to portfolios and LinkedIn - [Newser](https://www.newser.com/story/376599/frustrated-job-seekers-try-to-outsmart-ai-resume-bots.html)
- Ex-JPMorgan analyst joins AI-native bank OffDeal to automate pitchbooks and valuations - [Business Insider](https://www.businessinsider.com/left-jp-morgan-analyst-join-ai-investment-bank-startup-advice-2025-10)

## Chatbots on the Couch

- Survey: 70% of teens tried AI companions, Illinois bans independent AI therapy - [Her Campus](https://www.hercampus.com/school/toronto-mu/ai-as-your-therapist-the-future/)
- Chinese youth use DeepSeek and Doubao for mental health support as regulators warn of risks - [Rest of World](https://restofworld.org/2025/young-people-in-china-are-embracing-ai-therapy/)
- Experts warn AI chatbots over-validate and can deskill users, seek critique and remember they’re not human - [Channel News Asia](https://cnalifestyle.channelnewsasia.com/wellness/ai-validation-healthy-472241)

## Other News

- Apple nears deal to acquire Prompt AI after Seemour shutdown, xAI and Neuralink also explored bids - [PYMNTS](https://www.pymnts.com/apple/2025/apple-looks-to-acquire-tech-and-expertise-from-prompt-ai/)
- GitHub launches ‘EdgeAI for Beginners’: SLMs, optimization, multi-platform labs for private, real-time, cost-efficient local inference - [GitHub Gist](https://github.com/microsoft/edgeai-for-beginners)
- Apple’s AR glasses may run full visionOS, M5 iPad, new MacBook Pro, Vision Pro updates reportedly this week - [Bloomberg](https://www.bloomberg.com/news/newsletters/2025-10-12/inside-apple-s-glasses-pivot-when-apple-is-launching-m5-macbook-pro-ipad-pro)
- Police warn teens use Snapchat’s AI to fake ‘homeless man’ images for TikTok pranks, triggering 911 calls and risky responses - [The Verge](https://www.theverge.com/news/798681/police-stop-pulling-ai-homeless-man-tiktok-prank)
- Modi and Qualcomm CEO back India’s AI and semiconductor missions, deepen U.S.–India ties, discuss 6G transition - [Times of India](https://timesofindia.indiatimes.com/india/pm-modi-meets-qualcomm-ceo-discusses-indias-strides-in-ai-innovation-and-skilling/articleshow/124492155.cms)
- Aurora Innovation earns multiple Buy ratings and \\$12–\\$15 targets for autonomous long-haul trucking - [Yahoo](https://finance.yahoo.com/news/aurora-innovation-receives-series-buy-181413387.html)
- ARK Venture Fund offers retail access to 66 tech names, including OpenAI and SpaceX - [The Motley Fool](https://www.fool.com/investing/2025/10/12/open-ai-stock-spacex-stock-xai-stock-nvda-stock/)